### Data Preprocessing

### Importing Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

### Data Preprocessing Function

In [3]:
def load_and_clean_data(file_path):
    """
    Load and clean the data from the specified CSV file.

    Parameters:
    - file_path (str): Path to the CSV file containing the data.

    Returns:
    - DataFrame: Cleaned DataFrame containing selected columns with NaN values dropped.
    """
    df = pd.read_csv(file_path)
    df['Date received'] = pd.to_datetime(df['Date received'])
    
    cols_to_consider = ['Product', 'Sub-product', 'Issue', 'Sub-issue', 'Consumer complaint narrative',
                        'Company public response', 'Company', 'State', 'ZIP code', 'Date received']
    
    df_new = df[cols_to_consider]
    
    df_new = df_new.dropna()
    
    return df_new

In [4]:
def filter_by_years(df, years):
    """
    Filter the DataFrame to include only the rows corresponding to specified years.

    Parameters:
    - df (DataFrame): The DataFrame containing data to filter.
    - years (list of int): List of years to filter by.

    Returns:
    - DataFrame: Filtered DataFrame containing rows corresponding to specified years.
    """
    filtered_df = df[df['Date received'].dt.year.isin(years)].reset_index(drop=True)
    return filtered_df

In [5]:
def map_product_column(df):
    """
    Map values in the 'Product' column of the DataFrame to a standardized set of categories.

    Parameters:
    - df (DataFrame): The DataFrame containing the 'Product' column to map.

    Returns:
    - DataFrame: DataFrame with the 'Product' column values mapped to standardized categories.
    """

    product_map = {'Credit reporting or other personal consumer reports': 'Credit Reporting',
                   'Credit reporting, credit repair services, or other personal consumer reports': 'Credit Reporting',
                   'Payday loan, title loan, personal loan, or advance loan': 'Loans / Mortgage',
                   'Payday loan, title loan, or personal loan': 'Loans / Mortgage',
                   'Student loan': 'Loans / Mortgage',
                   'Vehicle loan or lease': 'Loans / Mortgage',
                   'Debt collection': 'Debt collection',
                   'Credit card or prepaid card': 'Credit/Prepaid Card',
                   'Credit card': 'Credit/Prepaid Card',
                   'Prepaid card': 'Credit/Prepaid Card',
                   'Mortgage': 'Loans / Mortgage',
                   'Checking or savings account': 'Checking or savings account'  
                  }
    # Map 'Product' column
    df.loc[:,'Product'] = df['Product'].map(product_map)
    
    return df


In [6]:
def clean_narrative(df):

    """
    Clean the consumer complaint narratives in the DataFrame by excluding short and irrelevant complaints.

    Parameters:
    - df (DataFrame): The input DataFrame containing consumer complaint data.

    Returns:
    - DataFrame: DataFrame with cleaned consumer complaint narratives.
    """
# Compute complaint length
    df['complaint length'] = df['Consumer complaint narrative'].apply(lambda x : len(x))

    df = df[df['complaint length'] > 20]
    
    complaints_to_exclude = ['See document attached', 'See the attached documents.', 'Incorrect information on my credit report', 'incorrect information on my credit report',
    'please see attached file','Please see documents Attached','Incorrect information on my credit report.', 'Please see attached file', 'see attached',
    'See attached', 'SEE ATTACHED DOCUMENTS', 'See Attached', 'SEE ATTACHMENT', 'SEE ATTACHMENTS', 
    'XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX']
    
    df = df[~df['Consumer complaint narrative'].isin(complaints_to_exclude)]
    return df

In [7]:
def filter_by_frequency(df):
    """
    Filter the DataFrame based on the frequency of sub-issues and sub-products.

    Parameters:
    - df (DataFrame): The input DataFrame containing consumer complaint data.

    Returns:
    - DataFrame: DataFrame filtered based on the frequency of sub-issues and sub-products.
    """
    # Select sub-issues with frequency > 500
    sub_issues_to_consider = df['Sub-issue'].value_counts()[df['Sub-issue'].value_counts() > 500].index

    # Filter DataFrame based on selected sub-issues
    reduced_subissues = df[df['Sub-issue'].isin(sub_issues_to_consider)]
    # Select sub-products with frequency > 100
    sub_products_to_consider = reduced_subissues['Sub-product'].value_counts()[reduced_subissues['Sub-product'].value_counts() > 100].index

    # Filter DataFrame based on selected sub-products
    final_df = reduced_subissues[reduced_subissues['Sub-product'].isin(sub_products_to_consider)]

    return final_df

In [8]:
def map_issue(df):
    """
    Map the issues to more defined and lesser number of issues  in the DataFrame.

    Parameters:
    - df (DataFrame): The input DataFrame containing consumer complaint data.

    Returns:
    - DataFrame: DataFrame with the 'Issue' column mapped to appropriate issues.
    """
    # Create a dictionary mapping issues to sub-issues
    issues_to_subissues = {}
    for issue in df['Issue'].value_counts().index:
        issues_to_subissues[issue] = list(df[df['Issue'] == issue]['Sub-issue'].value_counts().to_dict().keys())

    # Separate issues with only one sub-issue and more than one sub-issue
    one_subissue = {key: value for key, value in issues_to_subissues.items() if len(issues_to_subissues[key]) == 1}
    more_than_one_subissue = {key: value for key, value in issues_to_subissues.items() if len(issues_to_subissues[key]) > 1}

    # Existing issue mapping for issues with more than one sub-issue
    existing_issue_mapping = {issue: issue for issue in more_than_one_subissue}

    # Issue renaming based on provided mapping
    issue_renaming = {
        'Problem with a lender or other company charging your account': 'Account Operations and Unauthorized Transaction Issues',
        'Opening an account': 'Account Operations and Unauthorized Transaction Issues',
        'Getting a credit card': 'Account Operations and Unauthorized Transaction Issues',

        'Unable to get your credit report or credit score': 'Credit Report and Monitoring Issues',
        'Credit monitoring or identity theft protection services': 'Credit Report and Monitoring Issues',
        'Identity theft protection or other monitoring services': 'Credit Report and Monitoring Issues',

        'Problem caused by your funds being low': 'Payment and Funds Management',
        'Problem when making payments': 'Payment and Funds Management',
        'Managing the loan or lease': 'Payment and Funds Management',

        'False statements or representation': 'Disputes and Misrepresentations',
        'Fees or interest': 'Disputes and Misrepresentations',
        'Other features, terms, or problems': 'Disputes and Misrepresentations',

        'Took or threatened to take negative or legal action': 'Legal and Threat Actions'
    }

    # Combine issue renaming and existing issue mapping
    issues_mapping = {**issue_renaming, **existing_issue_mapping}

    # Map 'Issue' column using the defined mapping dictionary
    df['Issue'] = df['Issue'].apply(lambda x: issues_mapping[x])

    return df

In [9]:
def split_and_save_data(df,year, test_size=0.25, random_state=42, directory_to_save='./data_splits/'):
    """
    Split the input DataFrame into train and test sets, and save them as CSV files with the specified year included in the file names.

    Parameters:
    - df (DataFrame): The input DataFrame containing consumer complaint data.
    - year (int): The year associated with the data split.
    - test_size (float, optional): The proportion of the dataset to include in the test split. Default is 0.25.
    - random_state (int, optional): The seed used by the random number generator. Default is 42.
    - directory_to_save (str, optional): The directory path to save the data splits. Default is './data_splits/'.

    Returns:
    - None
    """
    # Split the data into train and test sets
    X = df['Consumer complaint narrative']
    y = df[['Product', 'Sub-product', 'Issue', 'Sub-issue']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y['Product'], test_size=test_size, random_state=random_state)

    # Concatenate X_train and y_train, and X_test and y_test respectively
    train_df = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
    test_df = pd.concat([X_test, y_test], axis=1).reset_index(drop=True)

    # Create directory if it doesn't exist
    if not os.path.exists(directory_to_save):
        os.makedirs(directory_to_save)
        
    # Save train and test data as CSV files with the year included in the file names
    train_df.to_csv(os.path.join(directory_to_save, f'train-data-split_{year}.csv'), index=False)
    test_df.to_csv(os.path.join(directory_to_save, f'test-data-split_{year}.csv'), index=False)

### Main Cleaning Pipeline

In [13]:
def main(file_path, year,year_name):
    # Load and clean the data
    df_cleaned = load_and_clean_data(file_path)
    
    # Filter the data by years
    df_filtered = filter_by_years(df_cleaned, year)
    
    # Map the 'Product' column
    df_mapped = map_product_column(df_filtered)
    
    # Clean the customer narratives in the data
    df_clean_narrative = clean_narrative(df_mapped)

    # Clean the data by frequency
    df_freq = filter_by_frequency(df_clean_narrative)

    #Mapping the Issues and filtering Sub Issues
    df_final = map_issue_to_subissue(df_freq)
    
    # Split and save the data
    split_and_save_data(df_final,year_name)
    return df_final

### Calling the data preprocessing script

In [14]:
file_path = 'complaints.csv'
years_to_include = [2023]
year_name=2023
df_final=main(file_path, years_to_include,year_name)

In [15]:
df_final.shape

(247517, 11)

In [16]:
df_final.Product.value_counts()

Credit Reporting               211695
Checking or savings account     12285
Credit/Prepaid Card             11975
Debt collection                  9380
Loans / Mortgage                 2182
Name: Product, dtype: int64

In [17]:
df_final['Sub-product'].value_counts()

Credit reporting                              210735
General-purpose credit card or charge card     10668
Checking account                               10409
Other debt                                      3041
I do not know                                   2316
Credit card debt                                1652
Federal student loan servicing                  1344
Store credit card                               1307
Medical debt                                    1053
Savings account                                  989
Other personal consumer report                   960
Loan                                             732
Other banking product or service                 725
Auto debt                                        581
Telecommunications debt                          419
Rental debt                                      179
CD (Certificate of Deposit)                      162
Mortgage debt                                    139
Conventional home mortgage                    

In [19]:
df_final['Issue'].value_counts()

Incorrect information on your report                                                87200
Improper use of your report                                                         61868
Problem with a credit reporting company's investigation into an existing problem    45371
Problem with a company's investigation into an existing problem                     20985
Managing an account                                                                  7367
Attempts to collect debt not owed                                                    5453
Problem with a purchase shown on your statement                                      3253
Account Operations and Unauthorized Transaction Issues                               2450
Written notification about debt                                                      2404
Disputes and Misrepresentations                                                      2311
Payment and Funds Management                                                         2259
Closing an

In [20]:
df_final['Sub-issue'].value_counts()

Information belongs to someone else                                                      57850
Reporting company used your report improperly                                            48732
Their investigation did not fix an error on your report                                  45395
Credit inquiries on your report that you don't recognize                                 13136
Account status incorrect                                                                 10208
Account information incorrect                                                             9267
Was not notified of investigation status or results                                       9200
Investigation took more than 30 days                                                      8928
Personal information incorrect                                                            5900
Debt is not yours                                                                         2785
Deposits and withdrawals                          